# Pré-processamento

In [21]:
import numpy as np
import pandas as pd

d = 10
hierarquias = [3, 1, 2]

# importando arquivos
doencas = pd.read_csv("dados/doencas.csv")
bg = pd.read_csv("dados/background.csv")

# anonimizando gênero, local e tratando data
def anon_genero(data):
    for i in data.index:
        if(data.at[i, 'genero'] == 'Female'):
            data.at[i, 'genero'] = 0
        else:
            data.at[i, 'genero'] = 1
                    
def anon_date(data, nivel):
    for i in data.index:
        data.at[i, 'data'] = data.at[i, 'data'].split('/')
        if(nivel == 1):
            data.at[i, 'data'][0] = '**'
        elif(nivel == 2):
            data.at[i, 'data'][1] = '**'
            data.at[i, 'data'][0] = '**'
        elif(nivel == 3):
            data.at[i, 'data'][2] = '****'
            data.at[i, 'data'][1] = '**'
            data.at[i, 'data'][0] = '**'
        
def anon_local(data, nivel):
    data.insert(0, 'localizacao', '*')
    for i in data.index:
        cid = data.at[i, 'cidade']
        est = data.at[i, 'estado']
        if(nivel == 1):
            cid = '*****'
        elif(nivel == 2):
            est = '*****'
        
        data.at[i, 'localizacao'] = [cid, est]
    data.drop(['cidade', 'estado'], axis=1, inplace=True)
    
def preprocess(data):
    new_data = data.copy()
    
    # removendo atributo identificador
    new_data.drop('id', axis=1, inplace=True)
    
    # generalizando localização e gênero, e tratando a data
    anon_local(new_data, 1)
    anon_genero(new_data)
    anon_date(new_data, 2)
    
    return new_data

doencas = preprocess(doencas)

doencas.head()

,localizacao,genero,data,doenca
0,"*****, New York",0,**/**/2000,Virose
1,"*****, Kansas",0,**/**/1999,Zika
2,"*****, New York",0,**/**/1999,Virose
3,"*****, New York",0,**/**/2000,Conjuntivite
4,"*****, New York",1,**/**/1999,Conjuntivite


## Verificando delta

In [17]:
def get_delta(data, bg, delta):
    
    # criando arrays de contadores e registros pra cada dataset
    data_registros = []
    data_cont = []
    bg_registros = []
    bg_cont = []
    
    for i in data:
        reg = [i[1], i[2], i[3]]
        cont = 1
        if(reg not in data_registros):
            data_registros.append(reg)
            data_cont.append(cont)
        else:
            for j in range(0, len(data_registros)):
                if(reg == data_registros[j]):
                    data_cont += 1

    for i in bg:
        reg = [i[1], i[2], i[3]]
        cont = 1
        if(reg not in bg_registros):
            bg_registros.append(reg)
            bg_cont.append(cont)
        else:
            for j in range(0, len(bg_registros)):
                if(reg == bg_registros[j]):
                    bg_cont[j] += 1
                    
    for i in range(0, len(bg)):
        if(data_registros[0] == bg_registros[i]):
            min = data_cont[0] / bg_cont[i]
            max = data_cont[0] / bg_cont[i] 
            
    for i in range(0, len(data_cont)):
        for j in range(0, len(bg_cont)):
            if(data_registros[i] == bg_registros[j]):
                prob = data_cont[i] / bg_cont[j]
                if(prob <= min):
                    min = prob
                else:
                    max = prob
    
    return (min >= delta[0] and max <= delta[1])


# # Função de delta-presença

In [ ]:
def delta_presenca(data, bg):
    
    

## Exportando dataset

In [ ]:
export_csv = doencas.to_csv(r'saida/doencas_' + str(d) + '.csv', index=None, header=True)